In [27]:
import pandas as pd
from datetime import datetime, timedelta

In [28]:
all_apes = pd.read_csv('../csvs/all_the_apes.csv')

In [29]:
all_listings = pd.read_csv('../all_listings.csv')

In [47]:
all_canc = pd.read_csv('../all_canc.csv')

In [74]:
all_canc.canc_event_time = all_canc.canc_event_time.astype("datetime64[ns]")

In [30]:
all_listings = all_listings.merge(all_apes, left_on='ape_id', right_on='ape_id', how='left')

In [31]:
all_sales = pd.read_csv('../all_sales.csv')
all_sales.sale_time = all_sales.sale_time.astype("datetime64[ns]")

In [32]:
all_sales = all_sales[all_sales.sale_time > datetime(2021, 10, 1)]

In [33]:
all_sales = all_sales.merge(all_apes, left_on='ape_id', right_on='ape_id', how='left')

In [34]:
all_listings.listing_event_time = all_listings.listing_event_time.astype("datetime64[ns]")

In [86]:
#all_listings[all_listings.listing_event_time == all_listings.listing_event_time.max()]
def get_max_listing(apes):
    return apes[apes.listing_event_time == apes.listing_event_time.max()]

In [87]:
recent_listings = all_listings.groupby('ape_id').apply(get_max_listing).reset_index(drop=True)

In [88]:
#all_listings[all_listings.listing_event_time == all_listings.listing_event_time.max()]
def get_max_canc(apes):
    return apes[apes.canc_event_time == apes.canc_event_time.max()]

In [89]:
recent_canc = all_canc.groupby('ape_id').apply(get_max_canc).reset_index(drop=True)

In [37]:
#calcualtes the mean sale for a ape mouths
total_mean = all_sales.sale_price.mean()

df_mouth = pd.DataFrame()

#loops through each mouth trait
for mouth in all_sales.Mouth.unique():

    #calcualte the rarity of trair as %
    rarity_perc = (all_apes[all_apes.Mouth == mouth].shape[0] / 10000) * 100

    #mean sale of that trait
    mean = all_sales[all_sales.Mouth == mouth].sale_price.mean()

    #create a object with trait, rarity and diff from avg
    df_mouth = df_mouth.append({
        'mouth': mouth,
        'mean': mean,
        'diff':  mean - total_mean,
        'rarity': rarity_perc
    }, ignore_index=True)

In [104]:
df_mouth

,diff,mean,mouth,rarity
0,-0.075341,0.784806,Grin Gold,3.09
1,-0.130552,0.729595,Snake,3.30
2,-0.015636,0.844511,Grillz,3.78
3,-0.046200,0.813947,Yawn,3.29
4,-0.100053,0.760094,Cigarette,3.23
5,0.054647,0.914794,Mustache,3.02
6,-0.032055,0.828092,Tongue Out,3.16
7,0.122524,0.982671,Flames,3.41
8,-0.056906,0.803241,Bubble Gum,2.80
9,NaN,NaN,NaN,0.00


In [105]:
cheapest_listing = all_listings[all_listings.Mouth == 'Covid Mask'].sort_values(by='listing_price', ascending=True).head(1)

#get the lowest listing price and compare to the mean

#if listings price is lower than mean, select


In [102]:
#check to make sure ape listing is not canc or sold
def is_still_listed(ape):
    ape_id = cheapest_listing.ape_id.item()
    ape_canc = recent_canc[recent_canc.ape_id == ape_id]

    has_canc = ape_canc.canc_event_time.item() > cheapest_listing.listing_event_time.item()
    
    ape_sale = all_sales[all_sales.ape_id == ape_id]

    has_sold = ape_sale.sale_time.item() > cheapest_listing.listing_event_time.item()

    if has_canc or has_sold:
        return False
    
    return True

In [106]:
is_still_listed(cheapest_listing)

False

In [107]:
cheapest_listing

,Unnamed: 0_x,ape_id,listing_event_id,listing_event_time,listing_price,Unnamed: 0_y,Clothes,Ears,Hat,Fur,Mouth,Eyes
9866,0,ApeGang #1448,1200491080,2021-10-02 04:33:00.999717,0.3,7190,NaN,Airpods,Cap,Brown,Covid Mask,High
